<a href="https://colab.research.google.com/github/aayu24/Kaggle/blob/master/CommonLitGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import os
# assert os.environ['COLAB_TPU_ADDR']

In [1]:
from google.colab import files
files.upload()         # expire any previous token(s) and upload recreated token

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aayushpatni","key":"ed56c47117dd39939581753865d001bc"}'}

In [2]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list

rm: cannot remove '/root/.kaggle': No such file or directory


In [3]:
!kaggle competitions download -c commonlit-evaluate-student-summaries

  0% 0.00/1.05M [00:00<?, ?B/s]
100% 1.05M/1.05M [00:00<00:00, 166MB/s]


In [4]:
!kaggle datasets download -d jonathanchan/deberta-v3-large

100% 768M/770M [00:08<00:00, 94.1MB/s]
100% 770M/770M [00:08<00:00, 94.2MB/s]


In [ ]:
!kaggle datasets download -d debarshichanda/debertav3base

In [ ]:
!kaggle datasets download -d aayushpatni/debertav3large-weights-for-commonlit

In [7]:
!mkdir debertav3base

In [5]:
!mkdir commonlit-evaluate-student-summaries

In [6]:
!mkdir dataset

In [7]:
!mv *.zip dataset/

In [8]:
!unzip dataset/commonlit-evaluate-student-summaries.zip -d commonlit-evaluate-student-summaries/

Archive:  dataset/commonlit-evaluate-student-summaries.zip
  inflating: commonlit-evaluate-student-summaries/prompts_test.csv  
  inflating: commonlit-evaluate-student-summaries/prompts_train.csv  
  inflating: commonlit-evaluate-student-summaries/sample_submission.csv  
  inflating: commonlit-evaluate-student-summaries/summaries_test.csv  
  inflating: commonlit-evaluate-student-summaries/summaries_train.csv  


In [ ]:
!unzip dataset/debertav3base.zip -d debertav3base/

In [13]:
!mkdir -p deberta-v3-large-raw

In [9]:
!unzip dataset/deberta-v3-large.zip -d deberta-v3-large-raw/

Archive:  dataset/deberta-v3-large.zip
  inflating: deberta-v3-large-raw/deberta-v3-large/README.md  
  inflating: deberta-v3-large-raw/deberta-v3-large/config.json  
  inflating: deberta-v3-large-raw/deberta-v3-large/pytorch_model.bin  
  inflating: deberta-v3-large-raw/deberta-v3-large/spm.model  
  inflating: deberta-v3-large-raw/deberta-v3-large/tokenizer_config.json  


In [ ]:
!unzip dataset/debertav3large-weights-for-commonlit.zip

In [ ]:
# !rm -rf /content/deberta-v3-large /content/debertav3base.zip /content/weights /content/v3_small_individual_targets /content/deberta-v3-large-freeze-6/

In [16]:
!rm -rf dataset

In [10]:
!pip install transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00


In [11]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

In [12]:
import numpy as np
import pandas as pd
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset
from transformers import TrainingArguments, Trainer
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold, GroupKFold
import gc

In [13]:
import random
import os
import warnings
import logging
import shutil
from tqdm import tqdm
from datasets import disable_progress_bar
warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
disable_progress_bar()
tqdm.pandas()

In [14]:
BASE_DIR = "/content"

In [15]:
CONFIG_MULTI = {
    "debertav3base1" : f"{BASE_DIR}/weights/weights/debertav3base",
    "debertav3large1" : f"{BASE_DIR}/deberta-v3-large/deberta-v3-large/deberta-v3-large",
    "debertav3large2" : f"{BASE_DIR}/deberta-v3-large-freeze-6/deberta-v3-large-freeze-6"
}

In [16]:
multi_models = ["debertav3base1","debertav3large1","debertav3large2"]

In [40]:
class CFG:
    # model_name="debertav3base"
    model_name="deberta-v3-large-raw/deberta-v3-large"
    learning_rate=1.5e-5 #1e-5
    weight_decay=0.02
    hidden_dropout_prob=0.007 #0.0
    attention_probs_dropout_prob=0.007 #0.0
    num_train_epochs=5
    n_splits=4
    batch_size=4 #1,4,8
    random_seed=42 #42,102
    save_steps=100 #500
    max_length=512 #1024
    n_freeze_layers=6 #6,10,12

In [18]:
# SEED 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(CFG.random_seed)

In [19]:
DATA_DIR = f"{BASE_DIR}/commonlit-evaluate-student-summaries/"

prompts_train = pd.read_csv(DATA_DIR + "prompts_train.csv")
summaries_train = pd.read_csv(DATA_DIR + "summaries_train.csv")
sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")

In [20]:
train = pd.merge(prompts_train,summaries_train,on='prompt_id')

In [21]:
gkf = GroupKFold(n_splits=CFG.n_splits) # Since 4 prompts in training set

for i, (_, val_index) in enumerate(gkf.split(train, groups=train["prompt_id"])):
    train.loc[val_index, "fold"] = i

In [22]:
train.groupby("fold").count()

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,content,wording
fold,,,,,,,,
0.0,2057,2057,2057,2057,2057,2057,2057,2057
1.0,2009,2009,2009,2009,2009,2009,2009,2009
2.0,1996,1996,1996,1996,1996,1996,1996,1996
3.0,1103,1103,1103,1103,1103,1103,1103,1103


## Pipeline components

In [23]:
tokenizer = AutoTokenizer.from_pretrained(f'{BASE_DIR}/{CFG.model_name}')

 Note that above tokenizer contains padding because for TPUs we want to have all the examples of same size, otherwise the tensors would have to be continouosly loaded again after each batch.

In [24]:
train_collator = DataCollatorWithPadding(tokenizer)

In [25]:
config = AutoConfig.from_pretrained(f'{BASE_DIR}/{CFG.model_name}')
config.update({
    "num_labels": 1, #2
    "problem_type": 'regression',
    "hidden_dropout_prob": CFG.hidden_dropout_prob,
    "attention_probs_dropout_prob": CFG.attention_probs_dropout_prob
})

In [26]:
config

DebertaV2Config {
  "_name_or_path": "/content/deberta-v3-large-raw/deberta-v3-large",
  "attention_probs_dropout_prob": 0.007,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.007,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "problem_type": "regression",
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.33.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

## Util functions

In [27]:
## Figure out generating labels columns for batch of thousand
def generate_tokens(examples: pd.DataFrame,mode='train',text_col='text'):
    encodings = tokenizer(examples[text_col],truncation=True,max_length=CFG.max_length,return_tensors='np')
    if mode == 'test':
        return encodings
    labels = np.column_stack((examples['content'],examples['wording']))
    return {**encodings, "labels": labels}

In [28]:
def generate_tokens_for_single_target(examples: pd.DataFrame,target: str,mode='train',text_col='text'):
    encodings = tokenizer(examples[text_col],truncation=True,max_length=CFG.max_length,return_tensors='np')
    if mode == 'test':
        return encodings
    labels = examples[target]
    return {**encodings, "labels": labels}

## Util functions - TPU

In [ ]:
## Figure out generating labels columns for batch of thousand
def generate_tokens(examples: pd.DataFrame,mode='train',text_col='text'):
    encodings = tokenizer(examples[text_col],truncation=True,max_length=CFG.max_length,padding='max_length',return_tensors='pt')
    if mode == 'test':
        return encodings
    labels = np.column_stack((examples['content'],examples['wording']))
    return {**encodings, "labels": labels}

In [ ]:
def generate_tokens_for_single_target(examples: pd.DataFrame,target: str,mode='train',text_col='text'):
    encodings = tokenizer(examples[text_col],truncation=True,max_length=CFG.max_length,padding='max_length',return_tensors='pt')
    if mode == 'test':
        return encodings
    labels = examples[target]
    return {**encodings, "labels": labels}

## Preparing text cols

In [29]:
sep = tokenizer.sep_token
train['full_text'] = train['prompt_title'] + sep + train['prompt_question'] + sep + train['text']

In [30]:
target_cols = ['content','wording']
text_col = 'full_text' #'text'
text_cols = [text_col]

## Metrics

In [31]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)

    return (content_score + wording_score)/2

## Inference

In [ ]:
# if os.path.exists(CFG.model_name):
#     shutil.rmtree(CFG.model_name)
# os.mkdir(CFG.model_name)

In [ ]:
# For the different target models
# for target in target_cols:
#     if os.path.exists(target):
#         shutil.rmtree(target)
#     os.mkdir(target)

GPU setup

In [33]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.empty_cache()
print(device)

cuda


In [34]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
for multi_model in multi_models:
    print(f'Model: {multi_model}')
    oof_preds = pd.DataFrame()
    for fold in range(CFG.n_splits):
        val_data = train[train['fold']==fold]
        val_dataset = Dataset.from_pandas(val_data[text_cols+target_cols])
        tokenized_val_dataset = val_dataset.map(lambda x: generate_tokens(x,text_col=text_col),batched=True)

        pretrained_model_dir = f"{CONFIG_MULTI[multi_model]}/fold_{fold}"
        model_dir = f"{multi_model}/fold_{fold}"
        model_fold_dir = os.path.join(model_dir, str(fold))

        model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_dir)
        model.eval() # Set default model mode to evaluation - https://huggingface.co/docs/transformers/main_classes/model
        model_gpu = model.to(device)

        test_args = TrainingArguments(
            output_dir=  model_fold_dir,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = CFG.batch_size,
            dataloader_drop_last = False,
            fp16=True
        )

        infer = Trainer(
            model = model_gpu,
            args = test_args,
            tokenizer = tokenizer,
            data_collator = train_collator
        )

        preds = infer.predict(tokenized_val_dataset)[0]
        train.loc[val_data.index,f"{multi_model}_content_pred"] = preds[:,0]
        train.loc[val_data.index,f"{multi_model}_wording_pred"] = preds[:,1]

        model_gpu.cpu()
        del model_gpu
        del model
        gc.collect()
        torch.cuda.empty_cache()

    print(f"Saving {multi_model} oof preds in csv file.")
    for target in target_cols:
        oof_preds[target] = train[f"{multi_model}_{target}_pred"]
    oof_preds['student_id'] = train['student_id']
    oof_preds.to_csv(f"{multi_model}_oof_preds.csv",index=False)

Model: debertav3base1


Saving debertav3base1 oof preds in csv file.
Model: debertav3large1


Saving debertav3large1 oof preds in csv file.
Model: debertav3large2


Saving debertav3large2 oof preds in csv file.


In [23]:
CONFIG_IND = {
    "debertav3base_ind_content" : f"{BASE_DIR}/v3_small_individual_targets/v3_small_individual_targets/content/debertav3base",
    "debertav3base_ind_wording" : f"{BASE_DIR}/v3_small_individual_targets/v3_small_individual_targets/wording/debertav3base",
    "debertav3large_6_ind_content" : f"{BASE_DIR}/content/deberta-v3-large-raw/deberta-v3-large",
    "debertav3large_6_ind_wording" : f"{BASE_DIR}/wording/deberta-v3-large-raw/deberta-v3-large"
}

In [24]:
ind_models = ["debertav3base_ind"]

In [ ]:
for ind_model in ind_models:
    print(f'Model: {ind_model}')
    oof_preds=pd.DataFrame()
    for target in target_cols:
      for fold in range(CFG.n_splits):
        val_data = train[train['fold']==fold]
        val_dataset = Dataset.from_pandas(val_data[text_cols+target_cols])
        tokenized_val_dataset = val_dataset.map(lambda x: generate_tokens_for_single_target(x,target,text_col=text_col),batched=True)
        pretrained_model_dir = f"{CONFIG_IND[f'{ind_model}_{target}']}/fold_{fold}"
        model_dir =  f"{target}/{ind_model}/fold_{fold}"
        model_fold_dir = os.path.join(model_dir, str(fold))

        model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_dir)
        model.eval() # Set default model mode to evaluation - https://huggingface.co/docs/transformers/main_classes/model
        model_gpu = model.to(device)

        test_args = TrainingArguments(
            output_dir=  model_fold_dir,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = CFG.batch_size,
            dataloader_drop_last = False,
            fp16=True
        )

        infer = Trainer(
            model = model_gpu,
            args = test_args,
            tokenizer = tokenizer,
            data_collator = train_collator
        )

        preds = infer.predict(tokenized_val_dataset)[0]
        train.loc[val_data.index,f"{ind_model}_{target}_pred"] = preds

        model_gpu.cpu()
        del model_gpu
        gc.collect()
        torch.cuda.empty_cache()
      oof_preds[target] = train[f"{ind_model}_{target}_pred"]
    print(f"Saving {ind_model} oof preds in csv file.")
    oof_preds['student_id'] = train['student_id']
    oof_preds.to_csv(f"{ind_model}_oof_preds.csv",index=False)


Model: debertav3base_ind


Saving debertav3base_ind oof preds in csv file.


In [ ]:
!mkdir commonlit-oof-preds

In [ ]:
!cp *.csv commonlit-oof-preds/

In [ ]:
!kaggle datasets init -p commonlit-oof-preds/

Data package template written to: commonlit-oof-preds/dataset-metadata.json


In [ ]:
!kaggle datasets create -p commonlit-oof-preds/

Starting upload for file debertav3base1_oof_preds.csv
100% 154k/154k [00:02<00:00, 63.7kB/s]
Upload successful: debertav3base1_oof_preds.csv (154KB)
Starting upload for file debertav3large1_oof_preds.csv
100% 158k/158k [00:02<00:00, 59.8kB/s]
Upload successful: debertav3large1_oof_preds.csv (158KB)
Starting upload for file debertav3base_ind_oof_preds.csv
100% 155k/155k [00:02<00:00, 53.3kB/s]
Upload successful: debertav3base_ind_oof_preds.csv (155KB)
Starting upload for file debertav3large2_oof_preds.csv
100% 154k/154k [00:02<00:00, 69.8kB/s]
Upload successful: debertav3large2_oof_preds.csv (154KB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/aayushpatni/commonlit-oof-preds


## Create different ensembles for wording and content

In [ ]:
scores = {}
for multi_model in multi_models:
  scores[multi_model] = compute_mcrmse((pd.read_csv(f'commonlit-oof-preds/{multi_model}_oof_preds.csv')[target_cols].values,train[target_cols].values))

In [ ]:
scores

{'debertav3base1': {'content_rmse': 0.476628584745796,
  'wording_rmse': 0.6357522646155996,
  'mcrmse': 0.5561904246806978},
 'debertav3large1': {'content_rmse': 0.6286460800064813,
  'wording_rmse': 0.6878538019328642,
  'mcrmse': 0.6582499409696727},
 'debertav3large2': {'content_rmse': 0.46874032364212037,
  'wording_rmse': 0.6064148516998585,
  'mcrmse': 0.5375775876709894}}

In [ ]:
for ind_model in ind_models:
  scores[ind_model] = compute_mcrmse((pd.read_csv(f'commonlit-oof-preds/{ind_model}_oof_preds.csv')[target_cols].values,train[target_cols].values))

In [ ]:
scores

{'debertav3base1': {'content_rmse': 0.476628584745796,
  'wording_rmse': 0.6357522646155996,
  'mcrmse': 0.5561904246806978},
 'debertav3large1': {'content_rmse': 0.6286460800064813,
  'wording_rmse': 0.6878538019328642,
  'mcrmse': 0.6582499409696727},
 'debertav3large2': {'content_rmse': 0.46874032364212037,
  'wording_rmse': 0.6064148516998585,
  'mcrmse': 0.5375775876709894},
 'debertav3base_ind': {'content_rmse': 0.5042827060349722,
  'wording_rmse': 0.611206661209505,
  'mcrmse': 0.5577446836222386}}

#### Hill Climbing

In [ ]:
scores_content = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1]['content_rmse'])}

In [ ]:
scores_content

{'debertav3large2': {'content_rmse': 0.46874032364212037,
  'wording_rmse': 0.6064148516998585,
  'mcrmse': 0.5375775876709894},
 'debertav3base1': {'content_rmse': 0.476628584745796,
  'wording_rmse': 0.6357522646155996,
  'mcrmse': 0.5561904246806978},
 'debertav3base_ind': {'content_rmse': 0.5042827060349722,
  'wording_rmse': 0.611206661209505,
  'mcrmse': 0.5577446836222386},
 'debertav3large1': {'content_rmse': 0.6286460800064813,
  'wording_rmse': 0.6878538019328642,
  'mcrmse': 0.6582499409696727}}

In [ ]:
scores_wording = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1]['wording_rmse'])}

In [ ]:
scores_wording

{'debertav3large2': {'content_rmse': 0.46874032364212037,
  'wording_rmse': 0.6064148516998585,
  'mcrmse': 0.5375775876709894},
 'debertav3base_ind': {'content_rmse': 0.5042827060349722,
  'wording_rmse': 0.611206661209505,
  'mcrmse': 0.5577446836222386},
 'debertav3base1': {'content_rmse': 0.476628584745796,
  'wording_rmse': 0.6357522646155996,
  'mcrmse': 0.5561904246806978},
 'debertav3large1': {'content_rmse': 0.6286460800064813,
  'wording_rmse': 0.6878538019328642,
  'mcrmse': 0.6582499409696727}}

In [ ]:
y = train['content'].values

In [ ]:
oof_df_content = pd.DataFrame()

for item in scores_content.items():
  oof_df_content[item[0]]=pd.read_csv(f'commonlit-oof-preds/{item[0]}_oof_preds.csv')['content'].values

In [ ]:
oof_df_content.head(2)

,debertav3large2,debertav3base1,debertav3base_ind,debertav3large1
0,-0.375244,-0.193848,-0.087341,-0.611328
1,-1.220703,-1.149414,-1.194336,-1.023438


In [ ]:
# Initialise
STOP = False
current_best_ensemble_content = oof_df_content.iloc[:,0]
# current_best_test_preds = test_preds.iloc[:,0]
MODELS = oof_df_content.iloc[:,1:]
weight_range = np.arange(0.01,0.51,0.01)   # or with negative weights: np.arange(-0.5,0.51,0.01)
history = [compute_metrics((current_best_ensemble_content,y))['rmse']]
i=0

# Hill climbing
while not STOP:
    i+=1
    potential_new_best_cv_score = compute_metrics((current_best_ensemble_content,y))['rmse']
    k_best, wgt_best = None, None
    for k in MODELS:
        for wgt in weight_range:
            potential_ensemble = (1-wgt) * current_best_ensemble_content + wgt * MODELS[k]
            cv_score = compute_metrics((potential_ensemble,y))['rmse']
            if cv_score < potential_new_best_cv_score:
                potential_new_best_cv_score = cv_score
                k_best, wgt_best = k, wgt

    if k_best is not None:
        current_best_ensemble_content = (1-wgt_best) * current_best_ensemble_content + wgt_best * MODELS[k_best]
        # current_best_test_preds = (1-wgt_best) * current_best_test_preds + wgt_best * test_preds[k_best]
        MODELS.drop(k_best, axis=1, inplace=True)
        if MODELS.shape[1]==0:
            STOP = True
        print(f'Iteration: {i}, Model added: {k_best}, Best weight: {wgt_best:.2f}, Best RMSE: {potential_new_best_cv_score:.5f}')
        history.append(potential_new_best_cv_score)
    else:
        STOP = True

Iteration: 1, Model added: debertav3base1, Best weight: 0.40, Best RMSE: 0.46218


In [ ]:
current_best_ensemble_content

0      -0.302686
1      -1.192187
2      -0.125220
3      -0.109253
4      -0.703223
          ...   
7160    1.583984
7161   -0.937695
7162    0.531543
7163   -0.116687
7164    0.814160
Length: 7165, dtype: float64

In [ ]:
oof_df_wording = pd.DataFrame()

for item in scores_wording.items():
  oof_df_wording[item[0]]=pd.read_csv(f'commonlit-oof-preds/{item[0]}_oof_preds.csv')['wording'].values

In [ ]:
oof_df_wording.head(2)

,debertav3large2,debertav3base_ind,debertav3base1,debertav3large1
0,0.038025,0.128784,0.056030,-0.393066
1,-1.045898,-1.162109,-1.103516,-0.826660


In [ ]:
# Initialise
STOP = False
current_best_ensemble_wording = oof_df_wording.iloc[:,0]
# current_best_test_preds = test_preds.iloc[:,0]
MODELS = oof_df_wording.iloc[:,1:]
weight_range = np.arange(0.01,0.51,0.01)   # or with negative weights: np.arange(-0.5,0.51,0.01)
history = [compute_metrics((current_best_ensemble_wording,y))['rmse']]
i=0

# Hill climbing
while not STOP:
    i+=1
    potential_new_best_cv_score = compute_metrics((current_best_ensemble_wording,y))['rmse']
    k_best, wgt_best = None, None
    for k in MODELS:
        for wgt in weight_range:
            potential_ensemble = (1-wgt) * current_best_ensemble_wording + wgt * MODELS[k]
            cv_score = compute_metrics((potential_ensemble,y))['rmse']
            if cv_score < potential_new_best_cv_score:
                potential_new_best_cv_score = cv_score
                k_best, wgt_best = k, wgt

    if k_best is not None:
        current_best_ensemble_wording = (1-wgt_best) * current_best_ensemble_wording + wgt_best * MODELS[k_best]
        # current_best_test_preds = (1-wgt_best) * current_best_test_preds + wgt_best * test_preds[k_best]
        MODELS.drop(k_best, axis=1, inplace=True)
        if MODELS.shape[1]==0:
            STOP = True
        print(f'Iteration: {i}, Model added: {k_best}, Best weight: {wgt_best:.2f}, Best RMSE: {potential_new_best_cv_score:.5f}')
        history.append(potential_new_best_cv_score)
    else:
        STOP = True

Iteration: 1, Model added: debertav3base1, Best weight: 0.50, Best RMSE: 0.58936
Iteration: 2, Model added: debertav3base_ind, Best weight: 0.35, Best RMSE: 0.58517
Iteration: 3, Model added: debertav3large1, Best weight: 0.08, Best RMSE: 0.58438


In [ ]:
current_best_ensemble_wording

0       0.038146
1      -1.083007
2      -0.219623
3       0.375772
4      -0.619894
          ...   
7160    0.373162
7161   -1.013632
7162   -0.293504
7163   -0.025156
7164    0.544369
Length: 7165, dtype: float64

In [ ]:
best_ensemble_preds_train = np.column_stack((current_best_ensemble_content,current_best_ensemble_wording))

In [ ]:
print(f'Best mcrmse: {compute_mcrmse((best_ensemble_preds_train,train[target_cols].values))}')

Best mcrmse: {'content_rmse': 0.4621805909691432, 'wording_rmse': 0.5984625324208646, 'mcrmse': 0.5303215616950039}


## Training debertav3large on individual columns

In [42]:
# For the different target models
for target in target_cols:
    if os.path.exists(target):
        shutil.rmtree(target)
    os.mkdir(target)

In [41]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
for target in target_cols:
    print(f"Target: {target}")
    for fold in range(CFG.n_splits):
        print(f"Fold: {fold}")
        fold_train_data = train[train['fold']!=fold]
        fold_val_data = train[train['fold']==fold]
        fold_train_dataset = Dataset.from_pandas(fold_train_data[text_cols + target_cols])
        fold_val_dataset = Dataset.from_pandas(fold_val_data[text_cols + target_cols])
        fold_train_tokenized = fold_train_dataset.map(lambda x: generate_tokens_for_single_target(x,target,text_col=text_col),batched=True)
        fold_val_tokenized = fold_val_dataset.map(lambda x: generate_tokens_for_single_target(x,target,text_col=text_col),batched=True)
        print(f"Number of training examples: {fold_train_tokenized.num_rows}")
        print(f"Number of validation examples: {fold_val_tokenized.num_rows}")
        gc.collect()

        model = AutoModelForSequenceClassification.from_pretrained(f'{BASE_DIR}/{CFG.model_name}',config=config)
        # freezing embeddings layer
        model.base_model.embeddings.requires_grad_(False)

        # freezing the initial N layers
        for k, param in model.base_model.encoder.layer.named_parameters():
            l = int(k.split(".")[0])
            if l < CFG.n_freeze_layers:
                param.requires_grad = False

        model_gpu = model.to(device)

        model_dir =  f"{target}/{CFG.model_name}/fold_{fold}"
        model_fold_dir = os.path.join(model_dir, str(fold))

        training_args = TrainingArguments(
            output_dir = model_fold_dir,
            report_to='none',
            load_best_model_at_end=True, # select best model
            learning_rate=CFG.learning_rate,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size*2,
            num_train_epochs=CFG.num_train_epochs,
            weight_decay=CFG.weight_decay,
            greater_is_better=False,
            metric_for_best_model="rmse", #mcrmse
            save_strategy='epoch', #steps
            evaluation_strategy='epoch',
            save_total_limit=1,
            gradient_accumulation_steps=4,
            # gradient_checkpointing=True,
            optim='adafactor',
            fp16=True,
            # save_steps = CFG.save_steps,
            # eval_steps = CFG.save_steps
        )

        trainer = Trainer(
            model = model_gpu,
            train_dataset = fold_train_tokenized,
            eval_dataset = fold_val_tokenized,
            args = training_args,
            data_collator = train_collator,
            tokenizer = tokenizer,
            compute_metrics = compute_metrics  #compute_mcrmse
        )

        trainer.train()
        trainer.save_model(model_dir)

##         Not needed since trainer saves everything - https://discuss.huggingface.co/t/what-is-the-purpose-of-save-pretrained/9167/2
#         model_gpu.save_pretrained(model_dir)
#         tokenizer.save_pretrained(model_dir)
        model_gpu.cpu()
        del model_gpu
        del model
        gc.collect()
        torch.cuda.empty_cache()

Target: content
Fold: 0
Number of training examples: 5108
Number of validation examples: 2057


Epoch,Training Loss,Validation Loss,Rmse
0,No log,0.190180,0.436096
1,0.269700,0.278995,0.528200
2,0.269700,0.200656,0.447946
4,0.129200,0.180339,0.424663
4,0.047200,0.214475,0.463114


Fold: 1
Number of training examples: 5156
Number of validation examples: 2009


Epoch,Training Loss,Validation Loss,Rmse
0,No log,0.617509,0.785817
1,0.258000,0.301197,0.548814
2,0.258000,0.283845,0.532771
4,0.130400,0.268790,0.518450
4,0.054700,0.279526,0.528702


Fold: 2
Number of training examples: 5169
Number of validation examples: 1996


...

Epoch,Training Loss,Validation Loss,Rmse
0,No log,0.482726,0.694785
1,0.421200,0.321872,0.567337
2,0.421200,0.304875,0.552154
4,0.208400,0.284974,0.533830
4,0.078600,0.290912,0.539363


Fold: 1
Number of training examples: 5156
Number of validation examples: 2009


Epoch,Training Loss,Validation Loss,Rmse
0,No log,0.799921,0.894383
1,0.389600,0.778880,0.882542
2,0.389600,0.765936,0.875178
4,0.182300,0.856752,0.925609
4,0.055800,0.789664,0.888630


Fold: 2
Number of training examples: 5169
Number of validation examples: 1996


Epoch,Training Loss,Validation Loss,Rmse
0,No log,0.273796,0.523256
1,0.430400,0.396022,0.629303


In [ ]:
# FLAGS={}
# xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

#### CV

In [ ]:
for target in target_cols:
    for fold in range(CFG.n_splits):
        val_data = train[train['fold']==fold]
        val_dataset = Dataset.from_pandas(val_data[text_cols+target_cols])
        tokenized_val_dataset = val_dataset.map(lambda x: generate_tokens_for_single_target(x,target,text_col=text_col),batched=True)

        model_dir =  f"{target}/{CFG.model_name}/fold_{fold}"
        model_fold_dir = os.path.join(model_dir, str(fold))

        model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        model.eval() # Set default model mode to evaluation - https://huggingface.co/docs/transformers/main_classes/model
        model_gpu = model.to(device)

        test_args = TrainingArguments(
            output_dir=  model_fold_dir,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = CFG.batch_size,
            dataloader_drop_last = False,
            fp16=True
        )

        infer = Trainer(
            model = model_gpu,
            args = test_args,
            tokenizer = tokenizer,
            data_collator = train_collator
        )

        preds = infer.predict(tokenized_val_dataset)[0]
        train.loc[val_data.index,f"{target}_pred"] = preds

        model_gpu.cpu()
        del model_gpu
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
print(compute_mcrmse((train[['content_pred','wording_pred']].values,train[target_cols].values)))

In [ ]:
oof_preds = pd.DataFrame()
for target in target_cols:
  oof_preds[target] = train[f"{target}_pred"]
oof_preds.to_csv(f'debertav3large3_oof_preds.csv',index=False)

## Next steps

*   setting dropout probs=0.0
*   full_text_2 = question + title + text helps.
*   once model with best score, using different random seeds (42,102)






